In [1]:
import sys

In [2]:
# General imports
import os
import json
import random
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt

# pytorch imports
import torch

# Transformer imports
from transformers import BertTokenizerFast
from transformers import BertModel

# Palmtree imports
from palmtree import dataset
from palmtree import model

In [4]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
base_path = "../../"
model_path = os.path.join(base_path, "models", "pretraining_model", "checkpoint-67246")
tokenizer_path = os.path.join(base_path, "tokenizer")
test_path = os.path.join(base_path, "datasets", "finetuning_dataset", "opcode_outlier_detection", "instructions_set.csv")
res_path = os.path.join(base_path, "results", "outlier_detection")

num_outlier_sets = 50000
num_runs = 10

In [6]:
opdcode_types = {
    "data_movement":      {"mov", "push", "pop", "cwtl", "cltq", "cqto","cqtd"},
    "unary_op":           {"inc", "dec", "neg", "not"},
    "binary_op":          {"add", "sub","imul", "xor", "or", "and", "lea", "leaq"},
    "shift_op":           {"sal", "sar", "shr", "shl"},
    "spacial_arith_op":   {"imulq", "mulq", "idivq", "divq"},
    "comparison_test":    {"cmp", "test"},
    "conditional_test":   {"sete", "setz", "setne", "setnz", "sets","setns", "setg", 
                           "setnle","setge", "setnl", "setl", "setnge","setle", "setng", 
                           "seta", "setnbe", "setae", "setnb", "setbe", "setna"},
    "jump":               {"jmp", "je", "jz", "jne", "jnz", "js", "jns", "jg", "jnle", 
                           "jge", "jnl", "jl", "jnge", "jle", "jng", "ja", "jnbe", "jae", 
                           "jnb", "jb", "jnae", "jbe", "jna"},
    "conditional_move":   {"cmove", "cmovz", "cmovne", "cmovenz", "cmovs", "cmovns", 
                           "cmovg", "cmovnle", "cmovge", "cmovnl", "cmovnge", "cmovle", 
                           "cmovng", "cmova", "cmovnbe", "cmovae", "cmovnb", "cmovb", 
                           "cmovnae", "cmovbe", "cmovna"},
    "call_ins":           {"call", "leave", "ret", "retn"},
    "float_point_arith":  {"fabs", "fadd", "faddp", "fchs", "fdiv", "fdivp", "fdivr", 
                           "fdivrp", "fiadd", "fidivr","fimul", "fisub", "fisubr", "fmul", 
                           "fmulp", "fprem", "fpreml", "frndint", "fscale", "fsqrt", 
                           "fsub", "fsubp", "fsubr", "fsubrp", "fxtract"}
}

In [8]:
def split_instructions_by_opcode_type(instruction_set):
    splitted_instructions = {}
    pbar = tqdm(instruction_set)
    pbar.set_description("Splitting Instructions by Opcode Type")
    for ins in pbar:
        # mnemonic = ins.split(" ")[0].strip().lower()
        mnemonic = instruction_set[ins].split(" ")[-1].strip().lower()
        for op_type in opdcode_types:
            if mnemonic in opdcode_types[op_type]:
                if op_type in splitted_instructions:
                    splitted_instructions[op_type].append(ins)
                else:
                    splitted_instructions[op_type] = [ins]
    return splitted_instructions

def split_instructions_by_operands_type(instruction_dict):
    splitted_instructions = {}
    pbar = tqdm(instruction_dict)
    pbar.set_description("Splitting Instructions by Operands Type")
    for instruction in pbar:
        op_type = instruction_dict[instruction]
        if op_type == "not-supported":
          continue
        if op_type in splitted_instructions:
            splitted_instructions[op_type].append(instruction)
        else:
            splitted_instructions[op_type] = [instruction]
    return splitted_instructions        

def create_outlier_detection_sets(instructions, num_sets, task):

    random.seed(datetime.now())
    
    if task == "opcode_outlier":
        splitted_instructions = split_instructions_by_opcode_type(instructions)
    elif task == "operands_outlier":
        splitted_instructions = split_instructions_by_operands_type(instructions)
    else:
        raise ValueError("Not supported Task")
        
    arr = list(splitted_instructions.keys())
    instruction_types = (arr * num_sets)[:num_sets]
    outlier_detection_sets = []
    pbar = tqdm(instruction_types)
    pbar.set_description("Creating Outlier Sets")
    for elem in pbar:
        instructions = splitted_instructions[elem]
        current_set = random.sample(instructions,4)
        type_outlier_list = set(instruction_types).difference(set([elem]))
        outlier_type = random.sample(type_outlier_list,1)[0]
        outlier_instructions = splitted_instructions[outlier_type]
        current_set.append(random.sample(outlier_instructions,1)[0])
        outlier_detection_sets.append(current_set)

    return outlier_detection_sets

In [9]:
def load_binbert_model(tokenizer_path, best_checkpoint):
    
    print("Loading Tokenizer ->", tokenizer_path)
    tokenizer =  BertTokenizerFast.from_pretrained(tokenizer_path)
    
    model = BertModel.from_pretrained(best_checkpoint, output_hidden_states=True)

    model.to("cuda")
    model.eval()

    return tokenizer,model

In [11]:
def binbert_batch_embeddings(all_batch_instructions, tokenizer, model):

  tokenized_instructions = tokenizer(all_batch_instructions, padding=True, truncation=True, max_length=512, return_tensors="pt")
  tokenized_instructions["input_ids"] = tokenized_instructions["input_ids"].to("cuda")
  tokenized_instructions["token_type_ids"] = tokenized_instructions["token_type_ids"].to("cuda")
  tokenized_instructions["attention_mask"] = tokenized_instructions["attention_mask"].to("cuda")
  masks = tokenized_instructions["attention_mask"]

  with torch.no_grad():
    outputs = model(**tokenized_instructions)
    layers_hidden_states = outputs.hidden_states

  return layers_hidden_states, masks

In [14]:
def outlier_detector(tokenizer, model, outlier_sets, set_batch_size, batch_embedding_function, num_hidden_layers, precomputed=False):

    cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

    detected = [0] * num_hidden_layers
    total = [0] * num_hidden_layers
    percentage = [0] * num_hidden_layers

    pbar = tqdm(list(range(len(outlier_sets)))[::set_batch_size])

    for i in pbar:

        batch_set = outlier_sets[i:i+set_batch_size]
        all_batch_instructions = [instr for instr_set in batch_set for instr in instr_set]
        
        if not precomputed:
            layers_hidden_states, masks = batch_embedding_function(all_batch_instructions, tokenizer, model)
        else:
            layers_hidden_states, masks = batch_embedding_function(all_batch_instructions, model)

        for l in range(0, num_hidden_layers):
            
            hidden_states = layers_hidden_states[l]

            for split_num, out_set in enumerate(torch.split(hidden_states, 5)):
                
                if masks is not None:
                    mask = masks[split_num*5:split_num*5+5]
                    denom = torch.sum(mask, -1, keepdim=True)
                    set_instructions_emb = torch.sum(out_set * mask.unsqueeze(-1), dim=1) / denom
                else:
                    set_instructions_emb = out_set
                
                emb_norm = set_instructions_emb/set_instructions_emb.norm(dim=1)[:, None]
                cos_sim = torch.mm(emb_norm, emb_norm.transpose(0,1))
                prova = cos(set_instructions_emb[0], set_instructions_emb[0])
                outlier = cos_sim.sum(1).min(dim=0)[1]

                if outlier == 4:
                    detected[l] += 1
                total[l] += 1
                percentage[l] = round(detected[l] / total[l], 4)

        pbar.set_description(" | ".join([f"Layer {i}: {p}" for i,p in enumerate(percentage)]))

        # Free Gpu Memory
        torch.cuda.empty_cache()

    return percentage

In [15]:
def save_res_table(mean, std, res_path, task):

  res_path = os.path.join(res_path, task)
  os.makedirs(res_path, exist_ok=True)
  res_filename = os.path.join(res_path, f"{checkpoint}.png")

  res = np.transpose([mean, std])
  df = pd.DataFrame(res, columns=["mean", "std"])

  fig, ax = plt.subplots()
  fig.patch.set_visible(False)
  ax.axis('off')
  ax.axis('tight')
  ax.table(cellText=df.values, colLabels=df.columns, loc='center')
  fig.tight_layout()
  plt.savefig(res_filename, dpi=100)

  plt.close(fig)

  return df

In [17]:
def binbert_outlier_detection(test_pd, tokenizer_path, model_path, num_outlier_sets, res_path, set_batch_size=64, num_runs=10, opcode=True, operand=True):

  vocab, model = load_binbert_model(tokenizer_path, model_path)
  
  if opcode:
      # Test for Opcode Outlier Detection
      runs_res = []
      # instruction_set = test_pd["olivetree_ins"].values
      instruction_set = {ins: op_type for ins, op_type in test_pd[["olivetree_ins", "mnemonic"]].values}

      for i in range(0, num_runs):
        print(f"-- RUN {i} --")
        opcode_outlier_sets = create_outlier_detection_sets(instruction_set, num_outlier_sets, "opcode_outlier")
        per_layer_percentages = outlier_detector(vocab, model, opcode_outlier_sets, set_batch_size, binbert_batch_embeddings, model.config.num_hidden_layers + 1)
        runs_res.append(per_layer_percentages)

      mean = np.mean(runs_res, axis=0)
      std = np.std(runs_res, axis=0)
      df = save_res_table(mean, std, res_path, "opcode_outlier")
      print(f"- Results For {num_runs} Runs -")
      print(df)

  if operand:
      # Test for Operand Outlier Detection
      runs_res = []
      instruction_dict = {ins: op_type for ins, op_type in test_pd[["olivetree_ins", "op_types"]].values}

      for i in range(0, num_runs):
        print(f"-- RUN {i} --")
        operand_outlier_sets = create_outlier_detection_sets(instruction_dict, num_outlier_sets, "operands_outlier")
        per_layer_percentages = outlier_detector(vocab, model, operand_outlier_sets, set_batch_size, binbert_batch_embeddings, model.config.num_hidden_layers + 1)
        runs_res.append(per_layer_percentages)

      mean = np.mean(runs_res, axis=0)
      std = np.std(runs_res, axis=0)
      df = save_res_table(mean, std, res_path, "operands_outlier")
      print(f"- Results For {num_runs} Runs -")
      print(df)

In [20]:
print("######## TESTING BinBert ######## ")
test_pd = pd.read_csv(test_path, sep="\t")
binbert_outlier_detection(test_pd, tokenizer_path, model_path, num_outlier_sets, res_path, set_batch_size = 512, num_runs = num_runs)

######## TESTING OLIVETREE ######## 
Loading Tokenizer -> /home/jovyan/work/olivetree/final_for_paper/tokenizer


Some weights of the model checkpoint at /home/jovyan/work/olivetree/final_for_paper/models/masked_language_model_only_bert_normal_mask30/checkpoint-95846 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /home/jovya

-- RUN 0 --


Creating Outlier Sets: 100%|██████████| 50000/50000 [00:20<00:00, 2412.78it/s]
Layer 0: 0.6858 | Layer 1: 0.7441 | Layer 2: 0.7482 | Layer 3: 0.7908 | Layer 4: 0.7783 | Layer 5: 0.7837 | Layer 6: 0.7732 | Layer 7: 0.7766 | Layer 8: 0.7436 | Layer 9: 0.7205 | Layer 10: 0.7182 | Layer 11: 0.7158 | Layer 12: 0.7252: 100%|██████████| 98/98 [03:23<00:00,  2.08s/it]


-- RUN 1 --


...

-- RUN 8 --


Creating Outlier Sets: 100%|██████████| 50000/50000 [00:20<00:00, 2412.66it/s]
Layer 0: 0.8921 | Layer 1: 0.8823 | Layer 2: 0.8678 | Layer 3: 0.8527 | Layer 4: 0.8601 | Layer 5: 0.8482 | Layer 6: 0.8342 | Layer 7: 0.8462 | Layer 8: 0.8409 | Layer 9: 0.8242 | Layer 10: 0.7878 | Layer 11: 0.7462 | Layer 12: 0.7529: 100%|██████████| 98/98 [03:24<00:00,  2.09s/it]


-- RUN 9 --


Creating Outlier Sets: 100%|██████████| 50000/50000 [00:20<00:00, 2399.66it/s]
Layer 0: 0.8905 | Layer 1: 0.8801 | Layer 2: 0.8665 | Layer 3: 0.8508 | Layer 4: 0.8577 | Layer 5: 0.8497 | Layer 6: 0.8351 | Layer 7: 0.8487 | Layer 8: 0.8414 | Layer 9: 0.8261 | Layer 10: 0.788 | Layer 11: 0.7474 | Layer 12: 0.7537: 100%|██████████| 98/98 [03:24<00:00,  2.09s/it] 


- Results For 10 Runs -
       mean       std
0   0.89136  0.001785
1   0.88176  0.002030
2   0.86753  0.001446
3   0.85196  0.001589
4   0.85984  0.001311
5   0.84912  0.001528
6   0.83360  0.001414
7   0.84693  0.001791
8   0.84075  0.001181
9   0.82473  0.000960
10  0.78791  0.001184
11  0.74686  0.001895
12  0.75307  0.001202
